In [ ]:
import numpy as np

 
import tensorflow as tf
import matplotlib.pyplot as plt
SEED_VALUE = 180
 
# Fix seed to make training deterministic.
np.random.seed(SEED_VALUE)
tf.random.set_seed(SEED_VALUE)

rng = np.random.default_rng(12345)

In [ ]:

step=10
low = -200
high = 200
size = int((high-low)/step)
off = rng.integers(low=-200, high=100, size=size)
off

In [ ]:
X_train = np.arange(low,high,step)
y_train = X_train * 2 + 5
y_train_lin = y_train
print(y_train)
y_train = y_train + off
y_train

In [ ]:

  
    
def plot_data(x1,y1,x2=[],y2=[],x3=[],y3=[]):
  plt.figure(figsize=(20,5))
  plt.scatter(x1,y1,color='green')
  if (len(x2) != 0):
      plt.scatter(x2,y2,color='blue')
  if (len(x3) != 0):
      plt.scatter(x3,y3,color='red')
        
  plt.xlabel('X')
  plt.ylabel('Y')
  plt.legend()
  plt.grid(True)

#import random
#rogue = np.arange(0,0,size)
#for i in range(1,10):
#    elem = random.randint(0,size-1)
#    off2 = random.randint(-31500,-31400)
#    y_train[elem] = y_train[elem]+off2

#x = lambda x : x if x > -200 else 0
#normalize = np.vectorize(x)
#y_train=normalize(y_train)

plot_data(X_train,y_train,X_train,y_train_lin)


In [ ]:
model=None
model = tf.keras.Sequential()
 
# Define the model consisting of a single neuron.
#model.add(Dense(units=1, input_shape=(1,)))
model.add(tf.keras.layers.Dense(1,input_shape=(1,)))
model.add(tf.keras.layers.Dense(100,activation='relu'))
model.add(tf.keras.layers.Dense(100,activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()
model.compile(optimizer=tf.keras.optimizers.Ftrl(),
              loss='mse',metrics=['MeanAbsoluteError'])


In [ ]:
history = model.fit(X_train, 
                    y_train, 
                    batch_size=1, 
                    epochs=100, 
                    validation_split=0.3,
                    verbose=2)

print("Done")

In [ ]:
def plot_history(history,data_names):
    n = 1
    plt.figure(n,figsize=(20,5))
    y_min = 1000000
    y_max = -100000
    for i in data_names:
        plt.plot(history.history[i],label=i)
        
        if (y_min > np.min(history.history[i])):
            y_min = np.min(history.history[i])

        if (y_max < np.max(history.history[i])):
            y_max = np.max(history.history[i])
            
        
        plt.xlim([0, len(history.history[i])])
        plt.ylim(y_min, y_max)
        plt.xlabel('Epoch')
        #plt.ylabel(i)
        n += 1

    plt.legend()
    plt.grid(True)
    plt.show()

plot_history(history,['loss','val_loss'])
#plot_history(history,['accuracy','val_accuracy'])

In [ ]:
model.evaluate(X_train,y_train_lin)

In [ ]:
x=[]
y=[]
x = np.arange(-200,400, step=7)
y = model.predict(x)
plot_data(X_train,y_train,x,y,X_train,y_train_lin)